In [ ]:
"""
Created on Mon Jan 13 16:00:47 2020
Group 7
@authors : G.H.
"""

In [37]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import numpy as np
import pandas as pd
import math
import operator
import os
from PIL import Image
from tqdm import tqdm_notebook as tqdm
import random

from shutil import copyfile

# keras : librairie de deep learning
import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, InputLayer, ReLU, AveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

# Lecure avec keras

In [70]:
# Reading of csv file SEATGURU annontate
csv_file = pd.read_csv('../g7_SEATGURU_annotate.csv', sep=';')
Airbus_file = csv_file.loc[csv_file['aircraft_manufacturer']  == 'Airbus',:]
Airbus_file = Airbus_file.loc[Airbus_file['view'] == 'Int']
Boeing_file = csv_file.loc[csv_file['aircraft_manufacturer']  == 'Boeing',:]
Boeing_file = Boeing_file.loc[Boeing_file['view'] == 'Int']
# List name pictures
list_img_Airbus = list(Airbus_file['name'])
list_img_Boeing = list(Boeing_file['name'])

In [71]:
path = './../All Data/ANALYSE IMAGE/IMG SEATGURU/'
new_path_train = './../G7_SEATGURU_cat/Int/Split_data_man/data_train'
new_path_val = './../G7_SEATGURU_cat/Int/Split_data_man/data_val'
new_paths = [new_path_train, new_path_val]

In [104]:
def split_train_test_Seatguru_cat(new_paths, path, list_planes = [list_img_Airbus, list_img_Boeing], 
                                  split_limit = .7, s = 8):
  

    for plane in list_planes:
        
        if plane == list_img_Airbus:
            classe = 'Airbus'
        if plane == list_img_Boeing:
            classe = 'Boeing'
        
        os.makedirs(new_paths[0] + '/' + classe, exist_ok = True)
        os.makedirs(new_paths[1]+ '/' + classe, exist_ok = True)
        
        random.seed(a = s)
        random.shuffle(plane)

        for pict in plane[:int(split_limit*len(plane))]:
            copyfile(path + '/' + pict, new_paths[0] + '/' + classe + '/' +  pict)
        for pict in plane[int(split_limit*len(plane)):]:
            copyfile(path + '/' + pict, new_paths[1] + '/' + classe + '/' +  pict)
        

In [105]:
# Creation of train and test samples
split_train_test_Seatguru_cat(new_paths, path)

In [106]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies


train_generator = train_datagen.flow_from_directory(new_path_train,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

test_generator = train_datagen.flow_from_directory(new_path_val,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 1123 images belonging to 2 classes.
Found 482 images belonging to 2 classes.


# Transfer learning

In [116]:
# create the base pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# add a global spatial average pooling layer
x = base_model.output


x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = MaxPooling2D(pool_size=(2, 2))(x)

# vector
x = Flatten()(x)

# let's add a fully-connected layer
x = Dense(1024)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(512)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(256)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)


# last layer which predict the class
predictions = Dense(2, activation = 'softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [115]:
# AveragePooling
step_size_train=train_generator.n//train_generator.batch_size

# Define learning rate reducer
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              patience=2, 
                              verbose=1, 
                              factor=0.5, 
                              min_lr=0.00001)

# How to include it in your training
model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    epochs=10,
                    validation_data=test_generator,
                    callbacks=[reduce_lr])  # here

Epoch 1/10
35/35 [==============================] - 227s 6s/step - loss: 0.1752 - accuracy: 0.9487 - val_loss: 2.9068 - val_accuracy: 0.5643
Epoch 2/10
35/35 [==============================] - 232s 7s/step - loss: 0.2171 - accuracy: 0.9432 - val_loss: 1.8595 - val_accuracy: 0.5975
Epoch 3/10
35/35 [==============================] - 232s 7s/step - loss: 0.1190 - accuracy: 0.9643 - val_loss: 2.1800 - val_accuracy: 0.5809
Epoch 4/10
35/35 [==============================] - 230s 7s/step - loss: 0.0809 - accuracy: 0.9707 - val_loss: 0.0257 - val_accuracy: 0.5809

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 5/10
35/35 [==============================] - 241s 7s/step - loss: 0.0369 - accuracy: 0.9927 - val_loss: 0.3659 - val_accuracy: 0.5913
Epoch 6/10
35/35 [==============================] - 244s 7s/step - loss: 0.0109 - accuracy: 1.0000 - val_loss: 5.0883 - val_accuracy: 0.6224
Epoch 7/10
35/35 [==============================] - 231s 7s/step - loss: 

In [117]:
# MaxPooling
step_size_train=train_generator.n//train_generator.batch_size

# Define learning rate reducer
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              patience=2, 
                              verbose=1, 
                              factor=0.5, 
                              min_lr=0.00001)

# How to include it in your training
model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    epochs=10,
                    validation_data=test_generator,
                    callbacks=[reduce_lr])  # here

Epoch 1/10
35/35 [==============================] - 225s 6s/step - loss: 0.8825 - accuracy: 0.5335 - val_loss: 2.7981 - val_accuracy: 0.5415
Epoch 2/10
35/35 [==============================] - 228s 7s/step - loss: 0.3948 - accuracy: 0.8323 - val_loss: 5.1430 - val_accuracy: 0.5270
Epoch 3/10
35/35 [==============================] - 242s 7s/step - loss: 0.1572 - accuracy: 0.9404 - val_loss: 0.7036 - val_accuracy: 0.5747
Epoch 4/10
35/35 [==============================] - 237s 7s/step - loss: 0.1389 - accuracy: 0.9496 - val_loss: 4.6922 - val_accuracy: 0.5498
Epoch 5/10
35/35 [==============================] - 232s 7s/step - loss: 0.2218 - accuracy: 0.9184 - val_loss: 1.3458 - val_accuracy: 0.5394

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 6/10
35/35 [==============================] - 233s 7s/step - loss: 0.1241 - accuracy: 0.9578 - val_loss: 0.7137 - val_accuracy: 0.5353
Epoch 7/10
35/35 [==============================] - 233s 7s/step - loss: 

In [118]:
# save the model
model.save('model_int_man.h5')